Names.csv 
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę w której wyliczysz wzrost w stopach (feet)
* Odpowiedz na pytanie jakie jest najpopularniesze imię?
* Dodaj kolumnę i policz wiek aktorów 
* Usuń kolumny (bio, death_details)
* Zmień nazwy kolumn - dodaj kapitalizaję i usuń _
* Posortuj dataframe po imieniu rosnąco

In [0]:
filePath = "dbfs:/FileStore/tables/Files/names.csv"
namesDf = spark.read.format("csv") \
              .option("header","true") \
              .option("inferSchema","true") \
              .load(filePath)

#display(namesDf)

In [0]:
from pyspark.sql.functions import * 

# Dodaję kolumnę z czasem, wysokość w stopach, wiek aktorów
namesDf = namesDf.withColumn('epoch', unix_timestamp(current_timestamp())) \
                 .withColumn('height_ft', round(col('height') * 0.0328084, 2)) \
                 .withColumn("date_of_birth", to_date("date_of_birth", "dd.mm.yyyy")) \
                 .withColumn("date_of_death", to_date("date_of_death", "dd.mm.yyyy")) \
                 .withColumn("age", year("date_of_death") - year("date_of_birth"))
display(namesDf.limit(5))

imdb_name_id name birth_name height bio birth_details date_of_birth place_of_birth death_details date_of_death place_of_death reason_of_death spouses_string spouses divorces spouses_with_children children epoch height_ft age nm0000001 Fred Astaire Frederic Austerlitz Jr. 177 Fred Astaire was born in Omaha, Nebraska, to Johanna (Geilus) and Fritz Austerlitz, a brewer. Fred entered show business at age 5. He was successful both in vaudeville and on Broadway in partnership with his sister, Adele Astaire. After Adele retired to marry in 1932, Astaire headed to Hollywood. Signed to RKO, he was loaned to MGM to appear in La danza di Venere (1933) before starting work on RKO's Carioca (1933). In the latter film, he began his highly successful partnership with Ginger Rogers, with whom he danced in 9 RKO pictures. During these years, he was also active in recording and radio. On film, Astaire later appeared opposite a number of partners through various studios. After a temporary retirement in 1945-7, during which he opened Fred Astaire Dance Studios, Astaire returned to film to star in more musicals through 1957. He subsequently performed a number of straight dramatic roles in film and TV. May 10, 1899 in Omaha, Nebraska, USA null Omaha, Nebraska, USA June 22, 1987 in Los Angeles, California, USA  (pneumonia) 1987-01-22 Los Angeles, California, USA   pneumonia Robyn Smith (27 June 1980 - 22 June 1987) (his death)Phyllis Livingston Potter (12 July 1933 - 13 September 1954) (her death) (2 children) 2 0 1 2 1744113142 5.81 null nm0000002 Lauren Bacall Betty Joan Perske 174 Lauren Bacall was born Betty Joan Perske on September 16, 1924, in New York City. She was the daughter of Natalie Weinstein-Bacal, a Romanian Jewish immigrant, and William Perske, who was born in New Jersey, to Polish Jewish parents. Her family was middle-class, with her father working as a salesman and her mother as a secretary. They divorced when she was five and she rarely saw her father after that.As a school girl, she originally wanted to be a dancer, but later switched gears to head into acting. She studied at the American Academy of Dramatic Arts in New York, after attending She was educated at Highland Manor, a private boarding school in Tarrytown, New York (through the generosity of wealthy uncles), and then at Julia Richman High School, which enabled her to get her feet wet in some off-Broadway productions.Out of school, she entered modeling and, because of her beauty, appeared on the cover of Harper's Bazaar, one of the most popular magazines in the US. The wife of famed director Howard Hawks spotted the picture in the publication and arranged with her husband to have Lauren take a screen test. As a result, which was entirely positive, she was given the part of Marie Browning in Acque del Sud (1944), a thriller opposite Humphrey Bogart, when she was just 19 years old. This not only set the tone for a fabulous career but also one of Hollywood's greatest love stories (she married Bogart in 1945). It was also the first of several Bogie-Bacall films.After 1945's L'agente confidenziale (1945), Lauren received second billing in Il grande sonno (1946) with Bogart. The mystery, in the role of Vivian Sternwood Rutledge, was a resounding success. Although she was making one film a year, each production would be eagerly awaited by the public. In 1947, again with her husband, Lauren starred in the thriller La fuga (1947). The film kept movie patrons on the edge of their seats. The following year, she starred with Bogart, Edward G. Robinson, and Lionel Barrymore in L'isola di corallo (1948). The crime drama was even more of a nail biter than her previous film.In 1950, Lauren starred in Le foglie d'oro (1950), a drama set in 1894. It was a film of note because she appeared without her husband - her co-star was Gary Cooper. In 1953, Lauren appeared in her first comedy as Schatze Page in Come sposare un milionario (1953). The film, with co-stars Marilyn Monroe and Betty Grab

In [0]:
# Najpopularniejsze imię 
most_popular_name = namesDf.withColumn("first_name", split("name", " ")[0]).select("name", "first_name").groupBy("first_name").count().sort(desc("count")).limit(1)
most_popular_name.explain(mode="formatted")
display(most_popular_name) # John

== Physical Plan ==
AdaptiveSparkPlan (7)
+- TakeOrderedAndProject (6)
   +- HashAggregate (5)
      +- Exchange (4)
         +- HashAggregate (3)
            +- Project (2)
               +- Scan csv  (1)


(1) Scan csv 
Output [1]: [name#21]
Batched: false
Location: InMemoryFileIndex [dbfs:/FileStore/tables/Files/names.csv]
ReadSchema: struct<name:string>

(2) Project
Output [1]: [split(name#21,  , 2)[0] AS first_name#213]
Input [1]: [name#21]

(3) HashAggregate
Input [1]: [first_name#213]
Keys [1]: [first_name#213]
Functions [1]: [partial_count(1) AS count#244L]
Aggregate Attributes [1]: [count#243L]
Results [2]: [first_name#213, count#244L]

(4) Exchange
Input [2]: [first_name#213, count#244L]
Arguments: hashpartitioning(first_name#213, 200), ENSURE_REQUIREMENTS, [plan_id=79]

(5) HashAggregate
Input [2]: [first_name#213, count#244L]
Keys [1]: [first_name#213]
Functions [1]: [finalmerge_count(merge count#244L) AS count(1)#239L]
Aggregate Attributes [1]: [count(1)#239L]
Results [2]:

first_name,count
John,873


In [0]:
# Usuwam kolumny bio i death_details
namesDf = namesDf.drop("bio", "death_details")


# Zmieniam nazwy kolumn, dodaję kapitalizację i usuwam _
def rename_column(col_name):
    return col_name.replace("_", " ").title().replace(" ", "")
  
new_columns = [rename_column(col) for col in namesDf.columns]
namesDf = namesDf.toDF(*new_columns)
display(namesDf.limit(5))

ImdbNameId,Name,BirthName,Height,BirthDetails,DateOfBirth,PlaceOfBirth,DateOfDeath,PlaceOfDeath,ReasonOfDeath,SpousesString,Spouses,Divorces,SpousesWithChildren,Children,Epoch,HeightFt,Age
nm0000001,Fred Astaire,Frederic Austerlitz Jr.,177,"May 10, 1899 in Omaha, Nebraska, USA",null,"Omaha, Nebraska, USA",1987-01-22,"Los Angeles, California, USA",pneumonia,Robyn Smith (27 June 1980 - 22 June 1987) (his death)Phyllis Livingston Potter (12 July 1933 - 13 September 1954) (her death) (2 children),2,0,1,2,1744113174,5.81,null
nm0000002,Lauren Bacall,Betty Joan Perske,174,"September 16, 1924 in The Bronx, New York City, New York, USA",1924-01-16,"The Bronx, New York City, New York, USA",2014-01-12,"New York City, New York, USA",stroke,Jason Robards (4 July 1961 - 10 September 1969) (divorced) (1 child)Humphrey Bogart (21 May 1945 - 14 January 1957) (his death) (2 children),2,1,2,3,1744113174,5.71,90
nm0000003,Brigitte Bardot,Brigitte Bardot,166,"September 28, 1934 in Paris, France",1934-01-28,"Paris, France",null,null,null,Bernard d'Ormale (16 August 1992 - present)Gunter Sachs (14 July 1966 - 3 July 1969) (divorced)Jacques Charrier (18 June 1959 - 20 November 1962) (divorced) (1 child)Roger Vadim (20 December 1952 - 6 December 1957) (divorced),4,3,1,1,1744113174,5.45,null
nm0000004,John Belushi,John Adam Belushi,170,"January 24, 1949 in Chicago, Illinois, USA",1949-01-24,"Chicago, Illinois, USA",1982-01-05,"Hollywood, Los Angeles, California, USA",acute cocaine and heroin intoxication,Judith Belushi-Pisano (31 December 1976 - 5 March 1982) (his death),1,0,0,0,1744113174,5.58,33
nm0000005,Ingmar Bergman,Ernst Ingmar Bergman,179,"July 14, 1918 in Uppsala, Uppsala län, Sweden",1918-01-14,"Uppsala, Uppsala län, Sweden",2007-01-30,"Fårö, Gotlands län, Sweden",natural causes,Ingrid Bergman (11 November 1971 - 20 May 1995) (her death) (1 child)Käbi Laretei (1959 - 1969) (divorced) (1 child)Gun Grut (1951 - 1959) (divorced) (1 child)Ellen Bergman (22 July 1945 - 1950) (divorced) (4 children)Else Fisher (25 March 1943 - 1945) (divorced) (1 child),5,4,5,8,1744113174,5.87,89


In [0]:
# Sortuję imieniem rosnąco
temp = namesDf.select('*').orderBy(namesDf["Name"].asc())
display(temp.limit(5))

ImdbNameId,Name,BirthName,Height,BirthDetails,DateOfBirth,PlaceOfBirth,DateOfDeath,PlaceOfDeath,ReasonOfDeath,SpousesString,Spouses,Divorces,SpousesWithChildren,Children,Epoch,HeightFt,Age
nm1001478,'Big' LeRoy Mobley,LeRoy King Mobley III,193,"April 1, 1973 in Atlantic City, New Jersey, USA",1973-01-01,"Atlantic City, New Jersey, USA",null,null,null,null,0,0,0,0,1744113190,6.33,null
nm0521811,'Ducky' Louie,Lawrence Louie,null,"July 22, 1931 in Berkeley, California, USA",1931-01-22,"Berkeley, California, USA",null,null,null,null,0,0,0,0,1744113190,null,null
nm0722372,'Little Billy' Rhodes,William H. Rhodes,null,"February 1, 1895 in Illinois, USA",null,"Illinois, USA",1967-01-24,"Hollywood, California, USA",stroke,null,0,0,0,0,1744113190,null,null
nm0946148,'Weird Al' Yankovic,Alfred Matthew Yankovic,183,"October 23, 1959 in Downey, California, USA",1959-01-23,"Downey, California, USA",null,null,null,Suzanne Krajewski (10 February 2001 - present) (1 child),1,0,1,1,1744113190,6.0,null
nm1265067,50 Cent,Curtis James Jackson III,183,"July 6, 1975 in Queens, New York City, New York, USA",1975-01-06,"Queens, New York City, New York, USA",null,null,null,null,0,0,0,0,1744113190,6.0,null


Movies.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę która wylicza ile lat upłynęło od publikacji filmu
* Dodaj kolumnę która pokaże budżet filmu jako wartość numeryczną, (trzeba usunac znaki walut)
* Usuń wiersze z dataframe gdzie wartości są null

In [0]:
filePath = "dbfs:/FileStore/tables/Files/movies.csv"
moviesDf = spark.read.format("csv") \
              .option("header","true") \
              .option("inferSchema","true") \
              .load(filePath)

display(moviesDf.limit(5))

imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey Depew",The adventures of a female reporter in the 1890s.,05.wrz,154,null,null,null,null,1.0,2.0
tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,26.12.1906,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Bella Cola, Will Coyne, Sam Crewes, Jack Ennis, John Forde, Vera Linden, Mr. Marshall, Mr. McKenzie, Frank Mills, Ollie Wilson",True story of notorious Australian outlaw Ned Kelly (1855-80).,06.sty,589,$ 2250,null,null,null,7.0,7.0
tt0001892,Den sorte drøm,Den sorte drøm,1911,19.08.1911,Drama,53,"Germany, Denmark",null,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helsengreen, Emil Albes, Hugo Flink, Mary Hagen","Two men of high rank are both wooing the beautiful and famous equestrian acrobat Stella. While Stella ignores the jeweler Hirsch, she accepts Count von Waldberg's offer to follow her home, ...",05.sie,188,null,null,null,null,5.0,2.0
tt0002101,Cleopatra,Cleopatra,1912,13.11.1912,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, Miss Robson, Helene Costello, Charles Sindelar, Mr. Howard, James R. Waite, Mr. Osborne, Harry Knowles, Mr. Paul, Mr. Brady, Mr. Corker",The fabled queen of Egypt's affair with Roman general Marc Antony is ultimately disastrous for both of them.,05.lut,446,$ 45000,null,null,null,25.0,3.0
tt0002130,L'Inferno,L'Inferno,1911,06.03.1911,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de Liguoro, Pier Delle Vigne, Augusto Milla, Attilio Motta, Emilise Beretta",Loosely adapted from Dante's Divine Comedy and inspired by the illustrations of Gustav Doré the original silent film has been restored and has a new score by Tangerine Dream.,7.0,2237,null,null,null,null,31.0,14.0


In [0]:
# Kolumna z wartością czasu wykonania notatnika w formacie epoch
moviesDf = moviesDf.withColumn("current_epoch_timestamp", unix_timestamp(current_timestamp()))

display(moviesDf.limit(5))

imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,current_epoch_timestamp
tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey Depew",The adventures of a female reporter in the 1890s.,05.wrz,154,null,null,null,null,1.0,2.0,1744114943
tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,26.12.1906,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Bella Cola, Will Coyne, Sam Crewes, Jack Ennis, John Forde, Vera Linden, Mr. Marshall, Mr. McKenzie, Frank Mills, Ollie Wilson",True story of notorious Australian outlaw Ned Kelly (1855-80).,06.sty,589,$ 2250,null,null,null,7.0,7.0,1744114943
tt0001892,Den sorte drøm,Den sorte drøm,1911,19.08.1911,Drama,53,"Germany, Denmark",null,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helsengreen, Emil Albes, Hugo Flink, Mary Hagen","Two men of high rank are both wooing the beautiful and famous equestrian acrobat Stella. While Stella ignores the jeweler Hirsch, she accepts Count von Waldberg's offer to follow her home, ...",05.sie,188,null,null,null,null,5.0,2.0,1744114943
tt0002101,Cleopatra,Cleopatra,1912,13.11.1912,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, Miss Robson, Helene Costello, Charles Sindelar, Mr. Howard, James R. Waite, Mr. Osborne, Harry Knowles, Mr. Paul, Mr. Brady, Mr. Corker",The fabled queen of Egypt's affair with Roman general Marc Antony is ultimately disastrous for both of them.,05.lut,446,$ 45000,null,null,null,25.0,3.0,1744114943
tt0002130,L'Inferno,L'Inferno,1911,06.03.1911,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de Liguoro, Pier Delle Vigne, Augusto Milla, Attilio Motta, Emilise Beretta",Loosely adapted from Dante's Divine Comedy and inspired by the illustrations of Gustav Doré the original silent film has been restored and has a new score by Tangerine Dream.,7.0,2237,null,null,null,null,31.0,14.0,1744114943


In [0]:
# Kolumna która wylicza ile lat upłynęło od publikacji filmu
moviesDf = moviesDf.withColumn("years_elapsed", lit(year(current_timestamp().cast("date")) - col("year")))

display(moviesDf.limit(5))

imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,current_epoch_timestamp,years_elapsed
tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey Depew",The adventures of a female reporter in the 1890s.,05.wrz,154,null,null,null,null,1.0,2.0,1744115028,131.0
tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,26.12.1906,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Bella Cola, Will Coyne, Sam Crewes, Jack Ennis, John Forde, Vera Linden, Mr. Marshall, Mr. McKenzie, Frank Mills, Ollie Wilson",True story of notorious Australian outlaw Ned Kelly (1855-80).,06.sty,589,$ 2250,null,null,null,7.0,7.0,1744115028,119.0
tt0001892,Den sorte drøm,Den sorte drøm,1911,19.08.1911,Drama,53,"Germany, Denmark",null,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helsengreen, Emil Albes, Hugo Flink, Mary Hagen","Two men of high rank are both wooing the beautiful and famous equestrian acrobat Stella. While Stella ignores the jeweler Hirsch, she accepts Count von Waldberg's offer to follow her home, ...",05.sie,188,null,null,null,null,5.0,2.0,1744115028,114.0
tt0002101,Cleopatra,Cleopatra,1912,13.11.1912,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, Miss Robson, Helene Costello, Charles Sindelar, Mr. Howard, James R. Waite, Mr. Osborne, Harry Knowles, Mr. Paul, Mr. Brady, Mr. Corker",The fabled queen of Egypt's affair with Roman general Marc Antony is ultimately disastrous for both of them.,05.lut,446,$ 45000,null,null,null,25.0,3.0,1744115028,113.0
tt0002130,L'Inferno,L'Inferno,1911,06.03.1911,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de Liguoro, Pier Delle Vigne, Augusto Milla, Attilio Motta, Emilise Beretta",Loosely adapted from Dante's Divine Comedy and inspired by the illustrations of Gustav Doré the original silent film has been restored and has a new score by Tangerine Dream.,7.0,2237,null,null,null,null,31.0,14.0,1744115028,114.0


In [0]:
# Kolumna która pokaże budżet filmu jako wartość numeryczną
from pyspark.sql.types import IntegerType
moviesDf = moviesDf.withColumn("budget", split(moviesDf["budget"], " ")[1].cast(IntegerType()))

display(moviesDf.limit(5))

imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,current_epoch_timestamp,years_elapsed
tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey Depew",The adventures of a female reporter in the 1890s.,05.wrz,154,null,null,null,null,1.0,2.0,1744115055,131.0
tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,26.12.1906,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Bella Cola, Will Coyne, Sam Crewes, Jack Ennis, John Forde, Vera Linden, Mr. Marshall, Mr. McKenzie, Frank Mills, Ollie Wilson",True story of notorious Australian outlaw Ned Kelly (1855-80).,06.sty,589,2250,null,null,null,7.0,7.0,1744115055,119.0
tt0001892,Den sorte drøm,Den sorte drøm,1911,19.08.1911,Drama,53,"Germany, Denmark",null,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helsengreen, Emil Albes, Hugo Flink, Mary Hagen","Two men of high rank are both wooing the beautiful and famous equestrian acrobat Stella. While Stella ignores the jeweler Hirsch, she accepts Count von Waldberg's offer to follow her home, ...",05.sie,188,null,null,null,null,5.0,2.0,1744115055,114.0
tt0002101,Cleopatra,Cleopatra,1912,13.11.1912,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, Miss Robson, Helene Costello, Charles Sindelar, Mr. Howard, James R. Waite, Mr. Osborne, Harry Knowles, Mr. Paul, Mr. Brady, Mr. Corker",The fabled queen of Egypt's affair with Roman general Marc Antony is ultimately disastrous for both of them.,05.lut,446,45000,null,null,null,25.0,3.0,1744115055,113.0
tt0002130,L'Inferno,L'Inferno,1911,06.03.1911,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de Liguoro, Pier Delle Vigne, Augusto Milla, Attilio Motta, Emilise Beretta",Loosely adapted from Dante's Divine Comedy and inspired by the illustrations of Gustav Doré the original silent film has been restored and has a new score by Tangerine Dream.,7.0,2237,null,null,null,null,31.0,14.0,1744115055,114.0


In [0]:
# Usuwam wiersze z dataframe gdzie wartości są null
moviesDf = moviesDf.dropna()

display(moviesDf.limit(5))

imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,current_epoch_timestamp,years_elapsed
tt0017136,Metropolis,Metropolis,1927,01.10.1928,"Drama, Sci-Fi",153,Germany,German,Fritz Lang,"Thea von Harbou, Thea von Harbou",Universum Film (UFA),"Alfred Abel, Gustav Fröhlich, Rudolf Klein-Rogge, Fritz Rasp, Theodor Loos, Erwin Biswanger, Heinrich George, Brigitte Helm","In a futuristic city sharply divided between the working class and the city planners, the son of the city's mastermind falls in love with a working class prophet who predicts the coming of a savior to mediate their differences.",08.mar,156076,6000000,$ 1236166,$ 1349711,98.0,495.0,208.0,1744115085,98.0
tt0021749,Luci della città,City Lights,1931,02.04.1931,"Comedy, Drama, Romance",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Virginia Cherrill, Florence Lee, Harry Myers, Al Ernest Garcia, Hank Mann, Charles Chaplin","With the aid of a wealthy erratic tippler, a dewy-eyed tramp who has fallen in love with a sightless flower girl accumulates money to be able to help her medically.",08.maj,162668,1500000,$ 19181,$ 46008,99.0,295.0,122.0,1744115085,94.0
tt0027977,Tempi moderni,Modern Times,1936,12.03.1937,"Comedy, Drama, Family",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Charles Chaplin, Paulette Goddard, Henry Bergman, Tiny Sandford, Chester Conklin, Hank Mann, Stanley Blystone, Al Ernest Garcia, Richard Alexander, Cecil Reynolds, Mira McKinney, Murdock MacQuarrie, Wilfred Lucas, Edward LeSaint, Fred Malatesta",The Tramp struggles to live in modern industrial society with the help of a young homeless woman.,08.maj,211250,1500000,$ 163577,$ 457688,96.0,307.0,115.0,1744115085,89.0
tt0029453,Il bandito della Casbah,Pépé le Moko,1937,22.10.1937,"Crime, Drama, Romance",94,France,"French, Arabic",Julien Duvivier,"Henri La Barthe, Henri La Barthe",Paris Film,"Jean Gabin, Gabriel Gabrio, Saturnin Fabre, Fernand Charpin, Lucas Gridoux, Gilbert Gil, Marcel Dalio, Charles Granval, Gaston Modot, René Bergeron, Paul Escoffier, Roger Legris, Jean Témerson, Robert Ozanne, Philippe Richard","A wanted gangster is both king and prisoner of the Casbah. He is protected from arrest by his friends, but is torn by his desire for freedom outside. A visiting Parisian beauty may just tempt his fate.",07.lip,6180,60000,$ 155895,$ 155895,98.0,46.0,55.0,1744115085,88.0
tt0029583,Biancaneve e i sette nani,Snow White and the Seven Dwarfs,1937,30.11.1938,"Animation, Family, Fantasy",83,USA,English,"William Cottrell, David Hand","Jacob Grimm, Wilhelm Grimm",Walt Disney Productions,"Roy Atwell, Stuart Buchanan, Adriana Caselotti, Eddie Collins, Pinto Colvig, Marion Darlington, Billy Gilbert, Otis Harlan, Lucille La Verne, James MacDonald, Scotty Mattraw, Moroni Olsen, Purv Pullen, Harry Stockwell, Bill Thompson","Exiled into the dangerous forest by her wicked stepmother, a princess is rescued by seven dwarf miners who make her part of their household.",07.cze,177157,1499000,$ 184925486,$ 184925486,95.0,260.0,173.0,1744115085,88.0


ratings.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dla każdego z poniższych wyliczeń nie bierz pod uwagę `nulls` 
* Kto daje lepsze oceny chłopcy czy dziewczyny dla całego setu
* Dla jednej z kolumn zmień typ danych do `long` 

In [0]:
filePath = "dbfs:/FileStore/tables/Files/ratings.csv"
ratingsDf = spark.read.format("csv") \
              .option("header","true") \
              .option("inferSchema","true") \
              .load(filePath)

display(ratingsDf.limit(5))

imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,votes_5,votes_4,votes_3,votes_2,votes_1,allgenders_0age_avg_vote,allgenders_0age_votes,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,allgenders_30age_votes,allgenders_45age_avg_vote,allgenders_45age_votes,males_allages_avg_vote,males_allages_votes,males_0age_avg_vote,males_0age_votes,males_18age_avg_vote,males_18age_votes,males_30age_avg_vote,males_30age_votes,males_45age_avg_vote,males_45age_votes,females_allages_avg_vote,females_allages_votes,females_0age_avg_vote,females_0age_votes,females_18age_avg_vote,females_18age_votes,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
tt0000009,5.9,154,5.9,6.0,12,4,10,43,28,28,9,1,5,14,7.2,4.0,6.0,38.0,5.7,50.0,6.6,35.0,6.2,97.0,7.0,1.0,5.9,24.0,5.6,36.0,6.7,31.0,6.0,35.0,7.3,3.0,5.9,14.0,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0
tt0000574,6.1,589,6.3,6.0,57,18,58,137,139,103,28,20,13,16,6.0,1.0,6.1,114.0,6.0,239.0,6.3,115.0,6.1,425.0,6.0,1.0,6.2,102.0,6.0,210.0,6.2,100.0,6.2,50.0,null,null,5.9,12.0,6.2,23.0,6.6,14.0,6.4,66.0,6.0,96.0,6.2,331.0
tt0001892,5.8,188,6.0,6.0,6,6,17,44,52,32,16,5,6,4,null,null,5.5,25.0,5.8,72.0,6.2,62.0,5.9,146.0,null,null,5.5,21.0,5.9,67.0,6.2,55.0,5.7,15.0,null,null,5.8,4.0,5.8,4.0,6.8,7.0,5.4,32.0,6.2,31.0,5.9,123.0
tt0002101,5.2,446,5.3,5.0,15,8,16,62,98,117,63,26,25,16,null,null,5.3,23.0,5.0,111.0,5.3,193.0,5.1,299.0,null,null,5.2,20.0,4.9,96.0,5.2,171.0,5.9,39.0,null,null,5.7,3.0,5.5,14.0,6.1,21.0,4.9,57.0,5.5,207.0,4.7,105.0
tt0002130,7.0,2237,6.9,7.0,210,225,436,641,344,169,66,39,20,87,7.5,4.0,7.0,402.0,7.0,895.0,7.1,482.0,7.0,1607.0,8.0,2.0,7.0,346.0,7.0,804.0,7.0,396.0,7.2,215.0,7.0,2.0,7.0,52.0,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0


In [0]:
#  Kolumna z wartością czasu wykonania notatnika w formacie epoch
ratingsDf = ratingsDf.withColumn("current_epoch_timestamp", unix_timestamp(current_timestamp()))

display(ratingsDf.limit(5))

imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,votes_5,votes_4,votes_3,votes_2,votes_1,allgenders_0age_avg_vote,allgenders_0age_votes,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,allgenders_30age_votes,allgenders_45age_avg_vote,allgenders_45age_votes,males_allages_avg_vote,males_allages_votes,males_0age_avg_vote,males_0age_votes,males_18age_avg_vote,males_18age_votes,males_30age_avg_vote,males_30age_votes,males_45age_avg_vote,males_45age_votes,females_allages_avg_vote,females_allages_votes,females_0age_avg_vote,females_0age_votes,females_18age_avg_vote,females_18age_votes,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes,current_epoch_timestamp
tt0000009,5.9,154,5.9,6.0,12,4,10,43,28,28,9,1,5,14,7.2,4.0,6.0,38.0,5.7,50.0,6.6,35.0,6.2,97.0,7.0,1.0,5.9,24.0,5.6,36.0,6.7,31.0,6.0,35.0,7.3,3.0,5.9,14.0,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0,1744115283
tt0000574,6.1,589,6.3,6.0,57,18,58,137,139,103,28,20,13,16,6.0,1.0,6.1,114.0,6.0,239.0,6.3,115.0,6.1,425.0,6.0,1.0,6.2,102.0,6.0,210.0,6.2,100.0,6.2,50.0,null,null,5.9,12.0,6.2,23.0,6.6,14.0,6.4,66.0,6.0,96.0,6.2,331.0,1744115283
tt0001892,5.8,188,6.0,6.0,6,6,17,44,52,32,16,5,6,4,null,null,5.5,25.0,5.8,72.0,6.2,62.0,5.9,146.0,null,null,5.5,21.0,5.9,67.0,6.2,55.0,5.7,15.0,null,null,5.8,4.0,5.8,4.0,6.8,7.0,5.4,32.0,6.2,31.0,5.9,123.0,1744115283
tt0002101,5.2,446,5.3,5.0,15,8,16,62,98,117,63,26,25,16,null,null,5.3,23.0,5.0,111.0,5.3,193.0,5.1,299.0,null,null,5.2,20.0,4.9,96.0,5.2,171.0,5.9,39.0,null,null,5.7,3.0,5.5,14.0,6.1,21.0,4.9,57.0,5.5,207.0,4.7,105.0,1744115283
tt0002130,7.0,2237,6.9,7.0,210,225,436,641,344,169,66,39,20,87,7.5,4.0,7.0,402.0,7.0,895.0,7.1,482.0,7.0,1607.0,8.0,2.0,7.0,346.0,7.0,804.0,7.0,396.0,7.2,215.0,7.0,2.0,7.0,52.0,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0,1744115283


In [0]:
# Nie biorę pod uwagę NULL
filtered_males_avg_col = ratingsDf.filter((col("males_allages_avg_vote").isNotNull()))
mean_male = filtered_males_avg_col.select(avg(col("males_allages_avg_vote")).alias("mean_male")).collect()[0]["mean_male"]

filtered_females_avg_col = ratingsDf.filter((col("females_allages_avg_vote").isNotNull()))
mean_female = filtered_females_avg_col.select(avg(col("females_allages_avg_vote")).alias("femean_male")).collect()[0]["femean_male"]

# Kto daje lepsze oceny chłopcy czy dziewczyny dla całego setu
if mean_male > mean_female:
  print("mezczyzni lepiej oceniaja")
else:
  print("kobiety lepiej oceniaja")

kobiety lepiej oceniaja


In [0]:
# Zmieniam typ jednej kolumny do long
from pyspark.sql.types import LongType
ratingsDf = ratingsDf.dropna().withColumn("total_votes", col("total_votes").cast(LongType()))

ratingsDf.dtypes

Out[22]: [('imdb_title_id', 'string'),
 ('weighted_average_vote', 'double'),
 ('total_votes', 'bigint'),
 ('mean_vote', 'double'),
 ('median_vote', 'double'),
 ('votes_10', 'int'),
 ('votes_9', 'int'),
 ('votes_8', 'int'),
 ('votes_7', 'int'),
 ('votes_6', 'int'),
 ('votes_5', 'int'),
 ('votes_4', 'int'),
 ('votes_3', 'int'),
 ('votes_2', 'int'),
 ('votes_1', 'int'),
 ('allgenders_0age_avg_vote', 'double'),
 ('allgenders_0age_votes', 'double'),
 ('allgenders_18age_avg_vote', 'double'),
 ('allgenders_18age_votes', 'double'),
 ('allgenders_30age_avg_vote', 'double'),
 ('allgenders_30age_votes', 'double'),
 ('allgenders_45age_avg_vote', 'double'),
 ('allgenders_45age_votes', 'double'),
 ('males_allages_avg_vote', 'double'),
 ('males_allages_votes', 'double'),
 ('males_0age_avg_vote', 'double'),
 ('males_0age_votes', 'double'),
 ('males_18age_avg_vote', 'double'),
 ('males_18age_votes', 'double'),
 ('males_30age_avg_vote', 'double'),
 ('males_30age_votes', 'double'),
 ('males_45age_avg_vot

# Zadanie 2

Spark UI to narzędzie do monitorowania i debugowania aplikacji Apache Spark. Oferuje kilka zakładek, które dostarczają różnych informacji o wykonywanych zadaniach:

- Jobs – pokazuje listę wszystkich zadań (jobs) w aplikacji, ich status (ukończone, w toku, nieudane), czas trwania i szczegółowy podział na etapy (stages). Pozwala analizować wydajność i identyfikować potencjalne wąskie gardła.

- Stages – zawiera szczegóły dotyczące poszczególnych etapów (stages) przetwarzania, w tym czas trwania, rozkład zadań (tasks) oraz ich statusy. Można tu także zobaczyć DAG (Directed Acyclic Graph) przedstawiający zależności między operacjami.

- Storage – prezentuje informacje o danych przechowywanych w pamięci i na dysku (np. RDD i DataFrames), ich rozmiar oraz partycje w poszczególnych węzłach klastra.

- Environment – zawiera szczegółowe informacje o konfiguracji środowiska Spark, w tym zmienne systemowe, ustawienia JVM oraz parametry aplikacji.

- Executors – pokazuje listę wykonawców (executors) wraz z informacjami o wykorzystaniu pamięci, liczbie przetworzonych zadań oraz ilości przesłanych danych między węzłami.

- SQL/DataFrame – jeśli aplikacja korzysta z Spark SQL, ta zakładka zawiera historię zapytań SQL, ich fizyczne i logiczne plany wykonania oraz statystyki wydajności.

- JDBC/ODBC Server - odnosi się do Spark Thrift Server, który umożliwia dostęp do Spark SQL za pomocą interfejsów JDBC (Java Database Connectivity) oraz ODBC (Open Database Connectivity).

- Structured Streaming - komponent Apache Spark, który umożliwia przetwarzanie danych strumieniowych w sposób deklaratywny, podobnie jak operacje na statycznych DataFrame'ach.

- Connect - Informacje odnośnie aktywnych sesji, ile są polaczone itd.

Dzięki Spark UI można analizować efektywność aplikacji, diagnozować błędy oraz optymalizować działanie Spark jobs.

# Zadanie 3

In [0]:
# PRZED GROUPBY
moviesDf.explain(mode="formatted")

== Physical Plan ==
* Project (3)
+- * Filter (2)
   +- Scan csv  (1)


(1) Scan csv 
Output [22]: [imdb_title_id#629, title#630, original_title#631, year#632, date_published#633, genre#634, duration#635, country#636, language#637, director#638, writer#639, production_company#640, actors#641, description#642, avg_vote#643, votes#644, budget#645, usa_gross_income#646, worlwide_gross_income#647, metascore#648, reviews_from_users#649, reviews_from_critics#650]
Batched: false
Location: InMemoryFileIndex [dbfs:/FileStore/tables/Files/movies.csv]
ReadSchema: struct<imdb_title_id:string,title:string,original_title:string,year:string,date_published:string,genre:string,duration:int,country:string,language:string,director:string,writer:string,production_company:string,actors:string,description:string,avg_vote:string,votes:int,budget:string,usa_gross_income:string,worlwide_gross_income:string,metascore:double,reviews_from_users:double,reviews_from_critics:double>

(2) Filter [codegen id : 1]
Inpu

In [0]:
# PO GROUPBY 
df_grouped = moviesDf.groupBy("year").agg(sum("duration"))
display(df_grouped.explain(mode="formatted"))

== Physical Plan ==
AdaptiveSparkPlan (7)
+- HashAggregate (6)
   +- Exchange (5)
      +- HashAggregate (4)
         +- Project (3)
            +- Filter (2)
               +- Scan csv  (1)


(1) Scan csv 
Output [22]: [imdb_title_id#629, title#630, original_title#631, year#632, date_published#633, genre#634, duration#635, country#636, language#637, director#638, writer#639, production_company#640, actors#641, description#642, avg_vote#643, votes#644, budget#645, usa_gross_income#646, worlwide_gross_income#647, metascore#648, reviews_from_users#649, reviews_from_critics#650]
Batched: false
Location: InMemoryFileIndex [dbfs:/FileStore/tables/Files/movies.csv]
ReadSchema: struct<imdb_title_id:string,title:string,original_title:string,year:string,date_published:string,genre:string,duration:int,country:string,language:string,director:string,writer:string,production_company:string,actors:string,description:string,avg_vote:string,votes:int,budget:string,usa_gross_income:string,worlwide_gros

Podczas operacji groupBy mamy dodatkowe kroki: HashAggregate, Exchange oraz AdaptiveSparkPlan.